In [27]:
import numpy as np 
import tensorflow as tf
import cv2 as cv
import numpy as np
from PIL import Image
import pickle
import matplotlib.pyplot as plt
import os
from IPython.display import display
import pandas as pd

In [28]:
path = "Final Data/posenet_mobilenet_v1_100_257x257_multi_kpt_stripped.tflite"
interpreter = tf.lite.Interpreter(model_path=path)
interpreter.allocate_tensors()

In [29]:
imgPath = 'Final Data/img data/'
imgNames = os.listdir('Final Data/img data/')
print(imgNames)

['walking.jpeg', 'walking 1.jpeg', 'walking 2.jpeg', 'walking 4.jpeg', 'walking 3.jpeg', 'cycling.jpeg', 'cycling 2.jpeg', 'cycling 3.jpeg', 'cycling 4.jpeg', 'cycling 1.jpeg']


In [30]:
def parse_output(heatmap_data,offset_data, threshold):

  '''
  Input:
    heatmap_data - hetmaps for an image. Three dimension array
    offset_data - offset vectors for an image. Three dimension array
    threshold - probability threshold for the keypoints. Scalar value
  Output:
    array with coordinates of the keypoints and flags for those that have
    low probability
  '''

  joint_num = heatmap_data.shape[-1]
  pose_kps = np.zeros((joint_num,3), np.uint32)

  for i in range(heatmap_data.shape[-1]):

      joint_heatmap = heatmap_data[...,i]
      max_val_pos = np.squeeze(np.argwhere(joint_heatmap==np.max(joint_heatmap)))
      remap_pos = np.array(max_val_pos/8*257,dtype=np.int32)
      pose_kps[i,0] = int(remap_pos[0] + offset_data[max_val_pos[0],max_val_pos[1],i])
      pose_kps[i,1] = int(remap_pos[1] + offset_data[max_val_pos[0],max_val_pos[1],i+joint_num])
      max_prob = np.max(joint_heatmap)

      if max_prob > threshold:
        if pose_kps[i,0] < 257 and pose_kps[i,1] < 257:
          pose_kps[i,2] = 1

  return pose_kps
def draw_kps(show_img,kps, ratio=None):
    for i in range(5,kps.shape[0]):
      if kps[i,2]:
        if isinstance(ratio, tuple):
          cv.circle(show_img,(int(round(kps[i,1]*ratio[1])),int(round(kps[i,0]*ratio[0]))),2,(0,255,255),round(int(1*ratio[1])))
          continue
        cv.circle(show_img,(kps[i,1],kps[i,0]),2,(0,255,255),6)
    return show_img

def imgProcessor(img_path, img_name, interpreter):
    img = cv.imread(img_path+img_name)
    print(img_path+img_name)
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    height = input_details[0]['shape'][1]
    width = input_details[0]['shape'][2]
    template_image = cv.resize(img, (width, height))
    template_input = np.expand_dims(template_image.copy(), axis=0)
    floating_model = input_details[0]['dtype'] == np.float32
    if floating_model:
        template_input = (np.float32(template_input) - 127.5) / 127.5
    interpreter.set_tensor(input_details[0]['index'], template_input)
    interpreter.invoke()
    template_output_data = interpreter.get_tensor(output_details[0]['index'])
    template_offset_data = interpreter.get_tensor(output_details[1]['index'])
    template_heatmaps = np.squeeze(template_output_data)
    template_offsets = np.squeeze(template_offset_data)
    print("template_heatmaps' shape:", template_heatmaps.shape)
    print("template_offsets' shape:", template_offsets.shape)
    template_show = np.squeeze((template_input.copy()*127.5+127.5)/255.0)
    template_show = np.array(template_show*255,np.uint8)
    template_kps = parse_output(template_heatmaps,template_offsets,0.3)
    kps = template_kps[5:]

    newPointImg = draw_kps(template_show.copy(),template_kps)
    cv.imwrite('testData/'+img_name, newPointImg)
    return kps

In [31]:
imgPath = 'Final Data/img data/'
imgNames = os.listdir('Final Data/img data/')
points_list = []
# scale = []
for i in imgNames:
    points_list.append(imgProcessor(imgPath, i, interpreter))
    

Final Data/img data/walking.jpeg
template_heatmaps' shape: (9, 9, 17)
template_offsets' shape: (9, 9, 34)
Final Data/img data/walking 1.jpeg
template_heatmaps' shape: (9, 9, 17)
template_offsets' shape: (9, 9, 34)
Final Data/img data/walking 2.jpeg
template_heatmaps' shape: (9, 9, 17)
template_offsets' shape: (9, 9, 34)
Final Data/img data/walking 4.jpeg
template_heatmaps' shape: (9, 9, 17)
template_offsets' shape: (9, 9, 34)
Final Data/img data/walking 3.jpeg
template_heatmaps' shape: (9, 9, 17)
template_offsets' shape: (9, 9, 34)
Final Data/img data/cycling.jpeg
template_heatmaps' shape: (9, 9, 17)
template_offsets' shape: (9, 9, 34)
Final Data/img data/cycling 2.jpeg
template_heatmaps' shape: (9, 9, 17)
template_offsets' shape: (9, 9, 34)
Final Data/img data/cycling 3.jpeg
template_heatmaps' shape: (9, 9, 17)
template_offsets' shape: (9, 9, 34)
Final Data/img data/cycling 4.jpeg
template_heatmaps' shape: (9, 9, 17)
template_offsets' shape: (9, 9, 34)
Final Data/img data/cycling 1.jp

# The Last column of every list points_lists represents the line the probability of that point in each image.
The lists of elements are represent like below: 
- leftShoulder 0
- rightShoulder 1
- leftElbow 2
- rightElbow 3
- leftWrist 4 
- rightWrist 5
- leftHip 6
- rightHip 7
- leftKnee 8 
- rightKnee 9 
- leftAnkle 10
- rightAnkle 11
Lets make a `csv` out of it to run inference of the models we trained on `models.ipynb`

In [32]:
points_list = np.asarray(points_list).astype('float32')
def containZero(x):
    if x[2] == 0: 
        return True
    return False
def dist(p1, p2):
    s = 0 
    if containZero(p1) or containZero(p2):
        return 0
    for i in range(len(p1)-1): 
        s+=(p1[i]-p2[i])**2
    return s**0.5
def saveCSV(kps):
    ankle_knee = [[],[]]
    knee_hip = [[],[]]
    wrist_elbow = [[],[]]
    elbow_shoulder = [[],[]]
    for i in kps:
        elbow_shoulder[0].append(dist(i[0], i[2]))
        elbow_shoulder[1].append(dist(i[1], i[3]))
        
        wrist_elbow[0].append(dist(i[4], i[2]))
        wrist_elbow[1].append(dist(i[5], i[3]))
        
        knee_hip[0].append(dist(i[6], i[8]))
        knee_hip[1].append(dist(i[7], i[9]))
        
        ankle_knee[0].append(dist(i[8], i[10]))
        ankle_knee[1].append(dist(i[9], i[11]))
    # no need to add scale as the image size is roughly (200,200)
    dict_13 = {'l ankle_knee': ankle_knee[0], 'r ankle_knee': ankle_knee[1], 'l knee_hip': knee_hip[0], 'r knee_hip': knee_hip[1],
             'l wrist_elbow':wrist_elbow[0], 'r wrist_elbow': wrist_elbow[1], 'l elbow_shoulder': elbow_shoulder[0], 'r elbow_shoulder': elbow_shoulder[1]
            }
    dict_high_Features = {'l leg': ankle_knee[0]+knee_hip[0], 'r leg': ankle_knee[1]+knee_hip[1],
                          'l hand': wrist_elbow[0]+elbow_shoulder[0], 'r hand': wrist_elbow[1]+elbow_shoulder[1]
                         }
    return pd.DataFrame(dict_13), pd.DataFrame(dict_high_Features) 

In [33]:
csv = saveCSV(points_list)
display(csv[0])
display(csv[1])

,l ankle_knee,r ankle_knee,l knee_hip,r knee_hip,l wrist_elbow,r wrist_elbow,l elbow_shoulder,r elbow_shoulder
0,42.579338,37.121422,38.470768,34.014703,14.212670,0.000000,25.942244,0.000000
1,33.015148,36.055513,44.147480,43.185646,30.016662,27.073973,33.541020,35.440090
2,0.000000,0.000000,38.052595,40.199502,0.000000,0.000000,0.000000,0.000000
3,25.000000,24.020824,32.249031,31.256999,22.360680,23.409400,18.681542,18.681542
4,30.016662,44.384682,0.000000,45.099889,25.495098,26.248809,30.083218,30.265492
5,0.000000,0.000000,38.052595,42.011903,0.000000,27.073973,29.546573,25.709920
6,0.000000,0.000000,0.000000,55.226805,30.413813,28.635642,26.248809,28.071338
7,29.614186,16.000000,28.792360,25.317978,15.231546,14.317821,22.360680,22.203603
8,0.000000,0.000000,46.010868,46.486557,27.658633,0.000000,25.079872,0.000000
9,55.443665,58.051701,42.201896,38.078866,0.000000,40.718546,0.000000,34.014703


,l leg,r leg,l hand,r hand
0,42.579338,37.121422,14.212670,0.000000
1,33.015148,36.055513,30.016662,27.073973
2,0.000000,0.000000,0.000000,0.000000
3,25.000000,24.020824,22.360680,23.409400
4,30.016662,44.384682,25.495098,26.248809
5,0.000000,0.000000,0.000000,27.073973
6,0.000000,0.000000,30.413813,28.635642
7,29.614186,16.000000,15.231546,14.317821
8,0.000000,0.000000,27.658633,0.000000
9,55.443665,58.051701,0.000000,40.718546


In [34]:
csv[0].to_csv('testData/test-8.csv', index = False)
csv[1].to_csv('testData/test-4.csv', index = False)